# Demo to download one file from NWM retrospective

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
import datetime
def generate_filename(year, month, day, hour, product, domain):
    # Create a datetime object from the input year, month, day and hour
    date = datetime.datetime(year, month, day, hour)

    # Create the filename using the strftime function to format the date, and the format function to insert the product and domain
    filename = "{}/{:%Y%m%d%H}00.{}_{}.comp".format(date.year, date, product, domain)

    return filename

# Example usage:
filename = generate_filename(1979, 2, 1, 4, 'CHRTOUT', 'DOMAIN1')
print(filename)

1979/197902010400.CHRTOUT_DOMAIN1.comp


In [10]:
!pip install boto3 xarray botocore -q

import boto3
import xarray as xr
from botocore import UNSIGNED
from botocore.client import Config

s3 = boto3.Session().client('s3', config=Config(signature_version=UNSIGNED))

bucket_name = 'noaa-nwm-retrospective-2-1-pds'
key = 'model_output/'+filename
savename = filename.split('/')[-1]

s3.download_file(bucket_name, key, savename)
ds = xr.open_dataset(savename)

In [11]:
ds

<xarray.Dataset>
Dimensions:         (time: 1, reference_time: 1, feature_id: 2776738)
Coordinates:
  * time            (time) datetime64[ns] 1979-02-01T04:00:00
  * reference_time  (reference_time) datetime64[ns] 1979-02-01
  * feature_id      (feature_id) int32 101 179 181 ... 1180001803 1180001804
    latitude        (feature_id) float32 ...
    longitude       (feature_id) float32 ...
Data variables:
    crs             |S1 ...
    order           (feature_id) int32 ...
    elevation       (feature_id) float32 ...
    streamflow      (feature_id) float64 ...
    q_lateral       (feature_id) float64 ...
    velocity        (feature_id) float64 ...
    qSfcLatRunoff   (feature_id) float64 ...
    qBucket         (feature_id) float64 ...
    qBtmVertRunoff  (feature_id) float64 ...
Attributes: (12/18)
    TITLE:                      OUTPUT FROM WRF-Hydro v5.2.0-beta2
    featureType:                timeSeries
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    model_initialization_time:  1979-02-01_00:00:00
    station_dimension:          feature_id
    model_output_valid_time:    1979-02-01_04:00:00
    ...                         ...
    model_configuration:        retrospective
    dev_OVRTSWCRT:              1
    dev_NOAH_TIMESTEP:          3600
    dev_channel_only:           0
    dev_channelBucket_only:     0
    dev:                        dev_ prefix indicates development/internal me...

In [12]:
to_be_select_feature_id = ds.feature_id.values
to_be_select_feature_id

array([       101,        179,        181, ..., 1180001802, 1180001803,
       1180001804])

In [13]:
i_id = 2
streamflow_df = ds.sel(feature_id=to_be_select_feature_id[i_id], method='nearest').to_dataframe()
streamflow_df.head()

,,crs,feature_id,latitude,longitude,order,elevation,streamflow,q_lateral,velocity,qSfcLatRunoff,qBucket,qBtmVertRunoff
time,reference_time,,,,,,,,,,,,
1979-02-01 04:00:00,1979-02-01,b'',181,46.016491,-67.998726,1,184.309998,0.02,0.0,0.14,0.0,0.00463,14.964001
